# Notebook 2 – Data Preprocessing  
## Hotel Booking Demand Analysis

Notebook này thực hiện bước **tiền xử lý dữ liệu (data preprocessing)** cho bộ dữ liệu *Hotel Booking Demand*,
nhằm làm sạch và chuẩn hóa dữ liệu trước khi sử dụng cho phân tích nâng cao
và các bước xây dựng mô hình học máy.


## Objectives

Mục tiêu chính của notebook này bao gồm:

- Loại bỏ các vấn đề về **chất lượng dữ liệu** đã được phát hiện trong giai đoạn EDA
- Chuẩn hóa **kiểu dữ liệu (data types)** để phản ánh đúng bản chất của từng biến
- Xử lý các **giá trị không hợp lệ** và **dữ liệu ngoại lệ rõ ràng**
- Xử lý **missing values** dựa trên bản chất và ngữ cảnh của từng biến
- Giảm độ thưa của dữ liệu bằng cách **gộp các danh mục hiếm**

**Kết quả đầu ra của notebook là:**
Một dataset đã được làm sạch và chuẩn hóa, sẵn sàng cho bước feature engineering và modeling


## Preprocessing Pipeline

Quy trình tiền xử lý dữ liệu được thực hiện theo thứ tự sau:

1. Load dữ liệu gốc
2. Loại bỏ các dòng dữ liệu trùng lặp
3. Chuẩn hóa và chuyển đổi kiểu dữ liệu
4. Xử lý các giá trị không hợp lệ (*invalid values*)
5. Xử lý missing values dựa trên bản chất từng biến
6. Gộp các danh mục hiếm trong các biến phân loại
7. Phát hiện và xử lý các outliers rõ ràng
8. Kiểm tra chất lượng dữ liệu trước khi lưu
9. Lưu clean dataset

## 1. Import Libraries and Load Data

In [14]:
import pandas as pd
import numpy as np
import sys 
from pathlib import Path 
import matplotlib.pyplot as plt
import seaborn as sns
import math

project_root = Path.cwd().parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.data.data_cleaner import (
    convert_data_types,
    handle_invalid_values,
    handle_missing_values,
    merge_rare_categories,
    validate_clean_data
)

from src.utils.data_quality import summarize_outliers_iqr

df = pd.read_csv("../data/raw/hotel_bookings.csv")
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


## 2. Remove Duplicate Rows
Các dòng dữ liệu trùng lặp không mang thêm thông tin và có thể làm sai lệch các thống kê cũng như mô hình học máy.  
Do đó, các bản ghi trùng lặp sẽ được kiểm tra và loại bỏ.

In [15]:
# Number of rows before removing duplicates
n_rows_before = df.shape[0]

# Remove duplicate rows
df = df.drop_duplicates()

# Number of rows after removing duplicates
n_rows_after = df.shape[0]

print(f"Removed {n_rows_before - n_rows_after} duplicate rows.")
print(f"Remaining rows: {n_rows_after}")

Removed 31994 duplicate rows.
Remaining rows: 87396


## 3. Data Type Conversion

Việc chuẩn hóa kiểu dữ liệu là bước quan trọng để đảm bảo mỗi biến được lưu trữ và xử lý đúng với bản chất của nó. Kiểu dữ liệu không phù hợp có thể làm khó phân tích, giảm hiệu quả tính toán và gây lỗi ở các bước xử lý tiếp theo.

Dựa trên EDA, các nhóm chuyển đổi chính gồm:
- **Biến thời gian**
    - `arrival_date_month` biểu diễn tháng đến, nhưng hiện đang ở dạng chuỗi
    (tên tháng) $\rightarrow$ dạng **số thứ tự tháng (1-12)** để thuận tiện cho phân tích định lượng và xử lý thời gian.

    - `reservation_status_date` là thông tin ngày, nhưng đang được lưu dưới dạng `object` $\rightarrow$ kiểu `datetime`.

- Các **biến phân loại** như hotel, meal, country, v.v $\rightarrow$ kiểu `category` để tối ưu bộ nhớ và làm rõ cấu trúc dữ liệu.

Việc chuyển đổi này **không làm thay đổi giá trị gốc của dữ liệu**, mà chỉ chuẩn hóa cách biểu diễn nhằm phục vụ tốt hơn cho các bước xử lý và phân tích tiếp theo.

In [16]:
df = convert_data_types(df)
df.dtypes

hotel                                   category
is_canceled                                int64
lead_time                                  int64
arrival_date_year                          int64
arrival_date_month                         int64
arrival_date_week_number                   int64
arrival_date_day_of_month                  int64
stays_in_weekend_nights                    int64
stays_in_week_nights                       int64
adults                                     int64
children                                 float64
babies                                     int64
meal                                    category
country                                 category
market_segment                          category
distribution_channel                    category
is_repeated_guest                          int64
previous_cancellations                     int64
previous_bookings_not_canceled             int64
reserved_room_type                      category
assigned_room_type  

Sau khi chuẩn hóa, kiểu dữ liệu của các biến đã phù hợp và nhất quán, sẵn sàng cho các bước preprocessing tiếp theo.

## 4. Invalid / Impossible Values Handling
Một số giá trị trong dataset tuy không bị thiếu nhưng không hợp lý theo ngữ cảnh đặt phòng, chẳng hạn như đặt phòng không có người lớn hoặc giá phòng âm. Những trường hợp này không phản ánh hành vi thực tế và có thể gây nhiễu cho phân tích.  
Do đó, các dòng dữ liệu có các giá trị không hợp lệ sẽ được loại bỏ hoàn toàn, vì không thể điều chỉnh hay suy diễn lại các giá trị này một cách hợp lý mà vẫn giữ được ý nghĩa ban đầu của dữ liệu.

In [17]:
n_rows_before = df.shape[0]

df = handle_invalid_values(df)

n_rows_after = df.shape[0]

print(f"Removed {n_rows_before - n_rows_after} rows with invalid values.")
print(f"Remaining rows: {n_rows_after}")

Removed 386 rows with invalid values.
Remaining rows: 87010


## 5. Missing Values Handling

Sau khi loại bỏ các giá trị không hợp lệ, dataset vẫn còn một số **giá trị bị thiếu**, chủ yếu xuất hiện ở các biến `agent`, `company`, `country` và `children`.  
Các giá trị thiếu này **không phân bố ngẫu nhiên** mà phản ánh đặc điểm thực tế của dữ liệu đặt phòng (ví dụ: nhiều booking không thông qua đại lý hoặc công ty).  
Do đó, missing values được xử lý **dựa trên bản chất của từng biến**, thay vì loại bỏ toàn bộ các dòng dữ liệu.  
Cụ thể:
- `agent`, `company`: trường hợp **không áp dụng (not applicable)** đối với các booking đặt trực tiếp $\rightarrow$ **giữ nguyên giá trị NaN**.
- `country`: thiếu thông tin quốc gia $\rightarrow$ **gán nhãn "Unknown"**.
- `children`: số lượng missing rất nhỏ, phân phối lệch $\rightarrow$ **điền median** để tránh ảnh hưởng bởi outliers.

In [18]:
df = handle_missing_values(df)
df.isna().sum()

hotel                                 0
is_canceled                           0
lead_time                             0
arrival_date_year                     0
arrival_date_month                    0
arrival_date_week_number              0
arrival_date_day_of_month             0
stays_in_weekend_nights               0
stays_in_week_nights                  0
adults                                0
children                              0
babies                                0
meal                                  0
country                               0
market_segment                        0
distribution_channel                  0
is_repeated_guest                     0
previous_cancellations                0
previous_bookings_not_canceled        0
reserved_room_type                    0
assigned_room_type                    0
booking_changes                       0
deposit_type                          0
agent                             12124
company                           81775


Sau bước này, các missing values quan trọng đã được xử lý phù hợp, trong khi các trường hợp *không áp dụng* vẫn được giữ lại để xử lý ở các bước sau nếu cần.

## 6. Rare Category Handling

Trong các biến phân loại, một số danh mục xuất hiện với tần suất rất thấp, đặc biệt ở các biến có số lượng danh mục lớn như `country`, `assigned_room_type`, `reserved_room_type` và `market_segment`.

Các danh mục hiếm này làm tăng độ thưa của dữ liệu và có thể dẫn đến overfitting trong quá trình xây dựng mô hình.

Ở bước này, các danh mục có tần suất thấp (dưới ngưỡng xác định) được gộp vào nhóm `"Other"` nhằm:
- Giảm số lượng mức phân loại
- Làm ổn định phân phối dữ liệu
- Giảm độ phức tạp cho các bước modeling

Lưu ý rằng các nhãn như `"Undefined"` và `"Unknown"` được xem là các trạng thái hợp lệ mang ý nghĩa riêng, do đó **không bị gộp** trong bước này.  
Ngoài ra, một số biến phân loại có ý nghĩa nghiệp vụ rõ ràng (ví dụ: `hotel`, `customer_type`, `deposit_type`) được bảo vệ và không áp dụng gộp danh mục hiếm.

In [19]:
# Select all categorical columns
categorical_cols = df.select_dtypes(include="category").columns

# Number of categories BEFORE merging
categories_before = {
    col: df[col].nunique()
    for col in categorical_cols
}

# Columns with clear business meaning -> do not merge
protected_cols = [
    "hotel",
    "customer_type",
    "deposit_type"
]

df = merge_rare_categories(
    df,
    categorical_cols=categorical_cols,
    threshold=0.01,
    protected_cols=protected_cols
)

print("Number of categories before vs after rare category merging:\n")

for col in categorical_cols:
    before = categories_before[col]
    after = df[col].nunique()
    print(f"{col:<25} {before:>2} -> {after:>2}")

Number of categories before vs after rare category merging:

hotel                      2 ->  2
meal                       5 ->  5
country                   178 -> 16
market_segment             8 ->  7
distribution_channel       5 ->  5
reserved_room_type         9 ->  7
assigned_room_type        11 ->  8
deposit_type               3 ->  3
customer_type              4 ->  4
reservation_status         3 ->  3


Việc gộp danh mục hiếm giúp giảm số lượng giá trị khác nhau ở các biến có nhiều nhóm như `country` và các biến loại phòng.  
Các biến còn lại hầu như không thay đổi, cho thấy việc gộp chỉ tác động đến những nhóm xuất hiện rất ít và không làm ảnh hưởng đến cấu trúc chung của dữ liệu.


## 7. Outlier Assessment

Sau khi các giá trị không hợp lệ đã được loại bỏ, dataset vẫn còn tồn tại một số giá trị lớn bất thường ở các biến số liên tục như `lead_time`, `adr` và `days_in_waiting_list`.

Các giá trị này không phản ánh lỗi dữ liệu, mà là những trường hợp hiếm nhưng có thể xảy ra trong thực tế. Do đó, chúng được xem là **outliers hợp lệ**.

Ở bước này, outliers được xác định và đánh giá nhằm đưa ra quyết định xử lý phù hợp trước khi lưu dataset đã làm sạch.

In [20]:
cols = [
    "lead_time",
    "stays_in_week_nights",
    "days_in_waiting_list",
    "adr"
]

outlier_df = summarize_outliers_iqr(df, cols)
outlier_df

,variable,outlier_count,outlier_ratio (%)
0,lead_time,2383,2.74
1,stays_in_week_nights,1511,1.74
2,days_in_waiting_list,855,0.98
3,adr,2508,2.88


Kết quả cho thấy tỷ lệ outliers ở các biến được phân tích dao động trong khoảng 1-3%.  
Các giá trị này chiếm tỷ trọng nhỏ và phản ánh các tình huống hiếm nhưng có thật trong dữ liệu đặt phòng.

Vì vậy, các outliers hợp lệ được **giữ nguyên** ở bước preprocessing để đảm bảo dữ liệu phân tích vẫn phản ánh đầy đủ thực tế.  
Việc giảm ảnh hưởng của các giá trị cực đoan sẽ được thực hiện ở giai đoạn feature engineering và modeling nếu cần.

## 8. Data Validation
Sau khi hoàn tất các bước tiền xử lý, dữ liệu được kiểm tra lại để đảm bảo không còn lỗi nghiêm trọng trước khi lưu tập dữ liệu sạch.  
Các kiểm tra bao gồm:
- Dataset không rỗng
- Các biến quan trọng không chứa giá trị thiếu
- Không tồn tại giá trị âm hoặc không hợp lệ
- Kiểu dữ liệu của các biến thời gian là nhất quán

Các điều kiện trên được xác nhận bằng các kiểm tra logic trực tiếp. Nếu vi phạm, pipeline sẽ dừng lại để tránh lan truyền lỗi sang các bước sau.

In [21]:
validate_clean_data(df)

Data validation passed. Dataset is clean and consistent.


## 9. Save Clean Dataset

Sau khi dữ liệu đã được làm sạch và kiểm tra tính hợp lệ, tập dữ liệu cuối cùng được lưu lại để sử dụng thống nhất trong các notebook tiếp theo.

Việc lưu dữ liệu ở bước này giúp:
- Đảm bảo tính tái lập của pipeline
- Tránh phải lặp lại toàn bộ quá trình preprocessing
- Tách biệt rõ ràng giữa bước làm sạch dữ liệu và các bước phân tích nâng cao

In [22]:
# Path to save clean dataset
output_path = "../data/processed/clean_data.csv"

# Save clean dataset
df.to_csv(output_path, index=False)

print(f"Clean dataset saved to {output_path}")

print("\nFinal clean dataset overview:")
df.info()

Clean dataset saved to ../data/processed/clean_data.csv

Final clean dataset overview:
<class 'pandas.core.frame.DataFrame'>
Index: 87010 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   hotel                           87010 non-null  category      
 1   is_canceled                     87010 non-null  int64         
 2   lead_time                       87010 non-null  int64         
 3   arrival_date_year               87010 non-null  int64         
 4   arrival_date_month              87010 non-null  int64         
 5   arrival_date_week_number        87010 non-null  int64         
 6   arrival_date_day_of_month       87010 non-null  int64         
 7   stays_in_weekend_nights         87010 non-null  int64         
 8   stays_in_week_nights            87010 non-null  int64         
 9   adults                          87010 non-null  int64  